In [33]:
import os
from pathlib import Path
import cv2

In [34]:
def valid_image(filename, target_name):
    return filename != target_name and (obj.endswith('.jpg') or obj.endswith('.jpeg') or obj.endswith('.png'))

In [35]:
def compare_histograms(target_path, collection_dir, outpath):
    target_name = os.path.split(target_path)[1]
    
    collection = [obj for obj in os.listdir(collection_dir) if valid_image(obj, target_name)]
    
    output = [['filename', 'distance']]
    
    
    
    
    
    print(collection)
    # load target_path as with cv2
    # generate normalized histogram of target
    # create empty output-list / dataframe that can be written to outpath as csv
    # add headers to the csv
    # create list of all files in collection_dir
    # filter off target from the list, if the target is inside the same dir
    # loop through all files in collection_dir and do:
        # load image with cv2
        # generate histogram and normalize
        # compare target histogram with the looped histogram
        # append the comparison value to the output-list
    # save the output list as a csv in outpath


In [36]:
target = os.path.join('..', 'data', 'img', 'davinci', 'Leonardo_da_Vinci_1.jpg')
collection_dir = os.path.join('..', 'data', 'img', 'davinci')

compare_histograms(target, collection_dir, './out')

NameError: name 'obj' is not defined